# Use Case - Stores, Hexbin and Distributor Relationship
## Part Two - Analysis

We will demonstrate some of the key geospatial functionality in Python using the data that we have previously prepared - primarily the store location data, market optimization data and distributor location data. We will look through some of the spatial analysis that we can conduct and aggregate our information in a way that supports the analysts with organized data ready for reporting or for feature engineering of a machine learning model. Additionally, the output of these datasets are exportable to AWS Athena, Aurora and Redshift for integration with other business analytics services. 

The spatial functionality in these notebooks can be done in-line with the ETL process, in order to simplify the end goal of extracting meaningful insights from geospatial data. In general, working with Python, R or Alteryx for geospatial modeling is far simpler and more reliable than in SQL. However, by pre-processing the data and doing the detailed modeling in the ETL phase, we can output data that is 'analyst-friendly'.  

### Use Case 

We will start by organizing our information in a way that simplifies the analytical process. Our store location, market optimization and distributor location data are all `POINT` spatial datasets. Additionally, we will bring in a low-resolution state dataset to provide a useful filter that assists in filtering data and reducing our computational space. 

Additionally, it is possible to extract and import data with AWS S3 using AWSwrangler (Python Package), establishing an efficient pipeline of building a geospatial repository. In this case, we will be working directly with GeoJSON files for upload (as Jupyter will recognize the spatial object in a GeoJSON file), and export as csv and Parquet for further use in AWS. 

In [3]:
# Library Import 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from numpy import cos, sin, arcsin, sqrt
from math import radians

# Remove Truncated cells
#pd.options.display.max_rows = 4000

In [5]:
gdf_store = gpd.read_file('../../01_Data/02_GeoJSON/store_loc.geojson')
gdf_market_op = gpd.read_file('../../01_Data/02_GeoJSON/market_op.geojson')
df_store_loc_attr = pd.read_csv('../../01_Data/04_CSV_Full/store_loc_attr.csv')
gdf_states = gpd.read_file('../../01_Data/02_GeoJSON/external_states.json')

### Data
We will start with the store location, the market optimziation data and the state boundaries. Note that all three datasets contain a column named `geometry`, the first two of which are `POINT` and the last consists of `POLYGONS` or `MULTIPOLYGONS`. 

One idea for analysis is to choose a buffer distance around each store, determine which hexbin centroids it contains, and then aggregate the resulting demographic data.  

In [8]:
gdf_store

,ID_Store,lat,lon,geometry
0,02553,39.312415,-76.751345,POINT (-76.75135 39.31242)
1,03112,36.749370,-95.936120,POINT (-95.93612 36.74937)
2,02869,35.592628,-87.052409,POINT (-87.05241 35.59263)
3,03712,41.098811,-73.442169,POINT (-73.44217 41.09881)
4,04199,40.062070,-80.597990,POINT (-80.59799 40.06207)
...,...,...,...,...
4513,04492,43.029702,-76.019877,POINT (-76.01988 43.02970)
4514,03419,45.439700,-122.750650,POINT (-122.75065 45.43970)
4515,03324,38.924922,-77.239797,POINT (-77.23980 38.92492)
4516,05161,38.673320,-90.458210,POINT (-90.45821 38.67332)


In [9]:
gdf_market_op

,hex_id,ID_Store,2020_PopTotal,2020_PopMillenial,2020_DaytimeWorkers,2020_SocialMediaLast30Days,2020_LifeModeGrp3,2020_FastFoodDeliv,2010_CollegeDorm,2010_PopUnder18,lat,lon,geometry
0,8829a1d229fffff,4014,3347.0,1027.0,1032.0,210.0,0.0,309.0,0.0,502.0,33.970754,-118.100893,POINT (-118.10089 33.97075)
1,8829a54957fffff,2682,1404.0,302.0,556.0,154.0,0.0,122.0,0.0,175.0,33.557801,-117.699743,POINT (-117.69974 33.55780)
2,8844c175ebfffff,2077,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,33.984820,-83.950115,POINT (-83.95012 33.98482)
3,8844dc202dfffff,2944,18.0,4.0,2.0,0.0,0.0,0.0,0.0,3.0,34.499830,-81.937439,POINT (-81.93744 34.49983)
4,8829a40cedfffff,2015,130.0,25.0,250.0,8.0,0.0,7.0,0.0,21.0,33.017830,-117.254753,POINT (-117.25475 33.01783)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
303005,882a100d43fffff,4141,10963.0,3965.0,6899.0,1121.0,2390.0,1258.0,0.0,1032.0,40.753770,-73.940931,POINT (-73.94093 40.75377)
303006,8829b6ca59fffff,1967,0.0,0.0,509.0,0.0,0.0,0.0,0.0,0.0,33.666821,-111.976708,POINT (-111.97671 33.66682)
303007,882af635e5fffff,756,1158.0,301.0,855.0,35.0,0.0,90.0,0.0,145.0,36.838091,-76.146246,POINT (-76.14625 36.83809)
303008,8826645717fffff,4220,0.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,41.669056,-87.698191,POINT (-87.69819 41.66906)


In [10]:
gdf_states.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ..."
1,0400000US25,25,Massachusetts,,7800.058,"MULTIPOLYGON (((-70.83204 41.60650, -70.82373 ..."
2,0400000US26,26,Michigan,,56538.901,"MULTIPOLYGON (((-88.68443 48.11579, -88.67563 ..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.05770 44.99743, -104.25015 44.9..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.05060 37.00040, -114.04999 36.9..."


# Spatial Join
One of the primary techniques in geospatial modeling is creating a spatial join, which is where we determine which objects intersect and overlap. For data reduction purposes, we can conduct a spatial join which will add a state location to each store, which simplifies our ability to filter and process data. We will repeat this process for both the store and the hexbins. While perhaps we could obtain a state from the store attribute table, the process of a spatial join eliminates the chance of manual data entry errors (accidentally typing MS for Missouri when they meant to type MO) and is consistent for both of our datasets. 

In [11]:
gdf_store = gpd.sjoin(gdf_store, gdf_states, how='inner')

In [12]:
gdf_store

,ID_Store,lat,lon,geometry,index_right,GEO_ID,STATE,NAME,LSAD,CENSUSAREA
0,02553,39.312415,-76.751345,POINT (-76.75135 39.31242),17,0400000US24,24,Maryland,,9707.241
14,01457,39.647772,-78.756858,POINT (-78.75686 39.64777),17,0400000US24,24,Maryland,,9707.241
107,03458,39.022797,-76.692822,POINT (-76.69282 39.02280),17,0400000US24,24,Maryland,,9707.241
112,03067,39.656750,-77.747160,POINT (-77.74716 39.65675),17,0400000US24,24,Maryland,,9707.241
157,01454,39.197620,-76.818540,POINT (-76.81854 39.19762),17,0400000US24,24,Maryland,,9707.241
...,...,...,...,...,...,...,...,...,...,...
2362,03414,42.785000,-71.507665,POINT (-71.50767 42.78500),41,0400000US33,33,New Hampshire,,8952.651
4386,01075,42.701850,-71.437340,POINT (-71.43734 42.70185),41,0400000US33,33,New Hampshire,,8952.651
3180,04843,41.831884,-71.424204,POINT (-71.42420 41.83188),10,0400000US44,44,Rhode Island,,1033.814
3980,03220,41.698803,-71.495978,POINT (-71.49598 41.69880),10,0400000US44,44,Rhode Island,,1033.814


In [13]:
gdf_market_op = gpd.sjoin(gdf_market_op, gdf_states, how='inner')

In [14]:
gdf_market_op

,hex_id,ID_Store,2020_PopTotal,2020_PopMillenial,2020_DaytimeWorkers,2020_SocialMediaLast30Days,2020_LifeModeGrp3,2020_FastFoodDeliv,2010_CollegeDorm,2010_PopUnder18,lat,lon,geometry,index_right,GEO_ID,STATE,NAME,LSAD,CENSUSAREA
0,8829a1d229fffff,4014,3347.0,1027.0,1032.0,210.0,0.0,309.0,0.0,502.0,33.970754,-118.100893,POINT (-118.10089 33.97075),22,0400000US06,06,California,,155779.22
1,8829a54957fffff,2682,1404.0,302.0,556.0,154.0,0.0,122.0,0.0,175.0,33.557801,-117.699743,POINT (-117.69974 33.55780),22,0400000US06,06,California,,155779.22
4,8829a40cedfffff,2015,130.0,25.0,250.0,8.0,0.0,7.0,0.0,21.0,33.017830,-117.254753,POINT (-117.25475 33.01783),22,0400000US06,06,California,,155779.22
42,8829a4764bfffff,2015,10.0,2.0,2.0,2.0,0.0,1.0,0.0,1.0,33.069157,-117.272497,POINT (-117.27250 33.06916),22,0400000US06,06,California,,155779.22
125,8829a188d7fffff,2509,3409.0,849.0,1000.0,292.0,0.0,295.0,0.0,392.0,34.216677,-118.511246,POINT (-118.51125 34.21668),22,0400000US06,06,California,,155779.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299536,882600b0ddfffff,4566,386.0,73.0,156.0,21.0,0.0,27.0,0.0,33.0,43.531408,-96.761517,POINT (-96.76152 43.53141),47,0400000US46,46,South Dakota,,75811.00
300290,882600b0cbfffff,4566,655.0,256.0,302.0,23.0,0.0,50.0,0.0,68.0,43.538480,-96.767749,POINT (-96.76775 43.53848),47,0400000US46,46,South Dakota,,75811.00
301121,882600b0a9fffff,4566,854.0,214.0,626.0,47.0,0.0,59.0,0.0,115.0,43.517731,-96.807024,POINT (-96.80702 43.51773),47,0400000US46,46,South Dakota,,75811.00
302181,882600b039fffff,4566,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,43.552984,-96.826623,POINT (-96.82662 43.55298),47,0400000US46,46,South Dakota,,75811.00


### Filter Data
One very important component of geospatial modeling is processing data in a way that is both effective and efficient. We want the right information, but we do not want to needlessly process data that increases cost and time (or that will fail due to time constraints). In this case, we have approximately 300k hexbins and 4500 stores. If we wanted to do a distance calculation on both of these datasets, we would have to calculate the distance between all 4500 stores to all 300k centroids, resulting in 1.3B calculations. However, now that we added the 'state' to both datasets, we can logically process the dataset, where each store only calculates against the hexbins in its state, exponentially reducing the time and computational cost of conducting this analysis. There is a possibility that a state falls within a border, but you can easily calculate whether the store's buffer distance crosses a state border, in which case you can then add in a constraint to also search for hexbin centroids within adjacent states. 

To keep things simple, we will do an analysis of store location and demographic information within Georgia. As you can see, this reduces the data to 357 stores and 30k hexbins, reducing the number of computations to a little over 10M. 

In [29]:
####-------- User Input ------- Set state to query 
state_query = 'Georgia'
max_filter_km = 35

gdf_market_op_query = gdf_market_op.loc[gdf_market_op.NAME == 'Georgia']
gdf_store_query = gdf_store.loc[gdf_store.NAME == 'Georgia']

In [11]:
# gdf_market_op_query = gdf_market_op_query.set_crs('EPSG:4326', allow_override = True)
# gdf_store = gdf_store.set_crs('EPSG:4326', allow_override = True)

In [18]:
cols = ['index_right', 'GEO_ID', 'STATE', 'NAME', 'LSAD', 'CENSUSAREA']

In [19]:
gdf_market_op_query = gdf_market_op_query.drop(columns = cols)

In [20]:
gdf_market_op_query

,hex_id,ID_Store,2020_PopTotal,2020_PopMillenial,2020_DaytimeWorkers,2020_SocialMediaLast30Days,2020_LifeModeGrp3,2020_FastFoodDeliv,2010_CollegeDorm,2010_PopUnder18,lat,lon,geometry
2,8844c175ebfffff,2077,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,33.984820,-83.950115,POINT (-83.95012 33.98482)
31,8844ea6235fffff,2111,31.0,7.0,5.0,0.0,0.0,2.0,0.0,2.0,33.706524,-85.019483,POINT (-85.01948 33.70652)
56,8844c12139fffff,686,168.0,44.0,48.0,11.0,0.0,11.0,0.0,25.0,34.037687,-84.220458,POINT (-84.22046 34.03769)
60,8844ce05c3fffff,2984,6.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,34.834672,-83.884863,POINT (-83.88486 34.83467)
61,8844c176b5fffff,1208,841.0,220.0,206.0,55.0,0.0,56.0,0.0,133.0,33.978858,-84.044630,POINT (-84.04463 33.97886)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302968,8844c128d9fffff,2242,244.0,61.0,253.0,19.0,0.0,17.0,0.0,40.0,34.106151,-84.178757,POINT (-84.17876 34.10615)
302971,8844ea6009fffff,2111,8.0,2.0,5.0,0.0,0.0,1.0,0.0,1.0,33.727074,-85.004219,POINT (-85.00422 33.72707)
302980,8844ce9939fffff,1225,386.0,119.0,82.0,10.0,0.0,26.0,0.0,65.0,34.256051,-83.844356,POINT (-83.84436 34.25605)
302990,8844d5a465fffff,643,279.0,100.0,229.0,3.0,0.0,7.0,0.0,37.0,32.468306,-81.776250,POINT (-81.77625 32.46831)


In [21]:
cols = ['ID_Store', 'lat', 'lon', 'geometry']
gdf_store_query = gdf_store_query[cols]

In [22]:
gdf_store_query

,ID_Store,lat,lon,geometry
21,00811,33.717738,-84.764489,POINT (-84.76449 33.71774)
23,00826,33.895100,-84.141001,POINT (-84.14100 33.89510)
34,04494,33.687922,-85.150197,POINT (-85.15020 33.68792)
45,02467,32.553896,-83.660591,POINT (-83.66059 32.55390)
67,80527,33.641080,-84.439531,POINT (-84.43953 33.64108)
...,...,...,...,...
4390,00817,34.760360,-84.997569,POINT (-84.99757 34.76036)
4396,01650,32.472023,-84.956275,POINT (-84.95628 32.47202)
4406,04786,32.138000,-81.226000,POINT (-81.22600 32.13800)
4458,00962,33.984357,-84.349916,POINT (-84.34992 33.98436)


### Methodology
There is different methodology that we can use
1. Create our own calculation, Haversine Distance (distance around a sphere, which in this case is Earth)
2. Use the buffer function

We will do both, but by creating our own calcuation adding this to the dataset, we give the analyst more flexibility in querying data down the line.  

For our own calculation, we will merge the two datasets, resulting in ~10M rows - a hexbin for each store in the state of Georgia. We will then calculate the distance from the store to the hexbin, filter out irrelevant values (for example, if we know that our buffers are all within 1-10 miles, we can remove hexbins that fall well outside this range). 

In [25]:
# Function to calculate distance for each row of the dataframe 
def haversine(row):
    lon1 = row['lon_x']
    lat1 = row['lat_x']
    lon2 = row['lon_y']
    lat2 = row['lat_y']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

In [26]:
gdf = gdf_store_query.merge(gdf_market_op_query, how = 'cross' )

In [27]:
gdf

,ID_Store_x,lat_x,lon_x,geometry_x,hex_id,ID_Store_y,2020_PopTotal,2020_PopMillenial,2020_DaytimeWorkers,2020_SocialMediaLast30Days,2020_LifeModeGrp3,2020_FastFoodDeliv,2010_CollegeDorm,2010_PopUnder18,lat_y,lon_y,geometry_y
0,00811,33.717738,-84.764489,POINT (-84.76449 33.71774),8844c175ebfffff,2077,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,33.984820,-83.950115,POINT (-83.95012 33.98482)
1,00811,33.717738,-84.764489,POINT (-84.76449 33.71774),8844ea6235fffff,2111,31.0,7.0,5.0,0.0,0.0,2.0,0.0,2.0,33.706524,-85.019483,POINT (-85.01948 33.70652)
2,00811,33.717738,-84.764489,POINT (-84.76449 33.71774),8844c12139fffff,686,168.0,44.0,48.0,11.0,0.0,11.0,0.0,25.0,34.037687,-84.220458,POINT (-84.22046 34.03769)
3,00811,33.717738,-84.764489,POINT (-84.76449 33.71774),8844ce05c3fffff,2984,6.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,34.834672,-83.884863,POINT (-83.88486 34.83467)
4,00811,33.717738,-84.764489,POINT (-84.76449 33.71774),8844c176b5fffff,1208,841.0,220.0,206.0,55.0,0.0,56.0,0.0,133.0,33.978858,-84.044630,POINT (-84.04463 33.97886)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10753906,80740,32.139190,-81.196900,POINT (-81.19690 32.13919),8844c128d9fffff,2242,244.0,61.0,253.0,19.0,0.0,17.0,0.0,40.0,34.106151,-84.178757,POINT (-84.17876 34.10615)
10753907,80740,32.139190,-81.196900,POINT (-81.19690 32.13919),8844ea6009fffff,2111,8.0,2.0,5.0,0.0,0.0,1.0,0.0,1.0,33.727074,-85.004219,POINT (-85.00422 33.72707)
10753908,80740,32.139190,-81.196900,POINT (-81.19690 32.13919),8844ce9939fffff,1225,386.0,119.0,82.0,10.0,0.0,26.0,0.0,65.0,34.256051,-83.844356,POINT (-83.84436 34.25605)
10753909,80740,32.139190,-81.196900,POINT (-81.19690 32.13919),8844d5a465fffff,643,279.0,100.0,229.0,3.0,0.0,7.0,0.0,37.0,32.468306,-81.776250,POINT (-81.77625 32.46831)


In [28]:
gdf['distance'] = gdf.apply(lambda row: haversine(row), axis=1)

In [30]:
cols = ['ID_Store_x', 'hex_id', '2020_PopTotal', '2020_PopMillenial', '2020_SocialMediaLast30Days',
        '2020_LifeModeGrp3', '2020_FastFoodDeliv', '2010_CollegeDorm', '2010_PopUnder18', 'distance'
       ]
gdf_store_demo = gdf[cols]

### Resulting DataFrame 
Now we have the resulting dataframe which is quite large, but instantly queryable. We can filter out the data that we know will not be relevant (in km), which we will set to 40km, or approximately 25 miles. the resulting dataframe is clean, concise and easy to aggregate in SQL, yet adaptable - the analyst can modify buffer distances real-time and it will aggregate data as it would in any other SQL query.  

In [ ]:
######### INSERT IMAGE OF AWS ATHENA!!!!!!!!!!!!

In [31]:
gdf_store_demo = gdf_store_demo.loc[gdf_store_demo.distance <= max_filter_km]

In [32]:
gdf_store_demo

,ID_Store_x,hex_id,2020_PopTotal,2020_PopMillenial,2020_SocialMediaLast30Days,2020_LifeModeGrp3,2020_FastFoodDeliv,2010_CollegeDorm,2010_PopUnder18,distance
1,00811,8844ea6235fffff,31.0,7.0,0.0,0.0,2.0,0.0,2.0,23.604036
7,00811,8844c1aaadfffff,1538.0,426.0,52.0,0.0,106.0,0.0,234.0,33.074202
12,00811,8844c1a42bfffff,372.0,92.0,19.0,0.0,29.0,0.0,40.0,15.125593
16,00811,8844c1a14dfffff,775.0,217.0,23.0,0.0,57.0,0.0,98.0,27.684947
23,00811,8844c1b151fffff,1403.0,453.0,63.0,0.0,108.0,0.0,27.0,29.169683
...,...,...,...,...,...,...,...,...,...,...
10753729,80740,8844d5d5b5fffff,285.0,73.0,11.0,0.0,19.0,0.0,24.0,16.740805
10753858,80740,8844d50325fffff,54.0,13.0,1.0,0.0,4.0,0.0,10.0,15.528914
10753880,80740,8844d5134dfffff,373.0,103.0,17.0,0.0,29.0,0.0,15.0,7.787622
10753899,80740,8844d5c561fffff,1375.0,300.0,171.0,0.0,138.0,0.0,141.0,13.908238


In [33]:
# Athena needs csv to not have a header
gdf_store_demo.to_csv("../../01_Data/03_CSV_Athena/store_demographics.csv", index = False, header = False)
gdf_store_demo.to_csv("../../01_Data/04_CSV_Full/store_demographics.csv", index = False, header = True)
gdf_states.to_csv('../../01_Data/04_CSV_Full/states_bounds.csv', index = False, header = True)

## Distributor and Store Coverage
Now that the first dataset is built and ready for analysis, there are some more interesting components when looking at distributors and store locations. For instance, what is the average distance of the closest 5 distributors to each store? Which stores overlap territory with others? Does this affect revenue?  

Let's work with Maryland, which has 146 store locations

In [45]:
####-------- User Input ------- Set state to query 
state_query = 'Maryland'
max_filter_km = 35

gdf_store_query_2 = gdf_store.loc[gdf_store.NAME == state_query]

In [46]:
gdf_store_query_2

,ID_Store,lat,lon,geometry,index_right,GEO_ID,STATE,NAME,LSAD,CENSUSAREA
0,02553,39.312415,-76.751345,POINT (-76.75135 39.31242),17,0400000US24,24,Maryland,,9707.241
14,01457,39.647772,-78.756858,POINT (-78.75686 39.64777),17,0400000US24,24,Maryland,,9707.241
107,03458,39.022797,-76.692822,POINT (-76.69282 39.02280),17,0400000US24,24,Maryland,,9707.241
112,03067,39.656750,-77.747160,POINT (-77.74716 39.65675),17,0400000US24,24,Maryland,,9707.241
157,01454,39.197620,-76.818540,POINT (-76.81854 39.19762),17,0400000US24,24,Maryland,,9707.241
...,...,...,...,...,...,...,...,...,...,...
4358,03540,38.980632,-76.536871,POINT (-76.53687 38.98063),17,0400000US24,24,Maryland,,9707.241
4379,03674,38.503569,-76.774135,POINT (-76.77414 38.50357),17,0400000US24,24,Maryland,,9707.241
4393,02129,39.015950,-76.930320,POINT (-76.93032 39.01595),17,0400000US24,24,Maryland,,9707.241
4403,01293,39.153200,-76.727120,POINT (-76.72712 39.15320),17,0400000US24,24,Maryland,,9707.241
